In [ ]:
from nbdev import *

In [ ]:
#default_exp dr

In [ ]:
#hide
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Dimensionality reduction

> This notebook gets the latent features from a multivariate time series 
given by an autoencoder and uses them as input for a dimensionality reduction algorithm

In [ ]:
#export
import umap
import pandas as pd
import numpy as np
from fastcore.all import *
from timecluster_extension.load import *
from timecluster_extension.utils import *

In [ ]:
#hide
import matplotlib.pyplot as plt
import wandb
from yaml import load, FullLoader
from tensorflow.keras.models import load_model
import hdbscan

## Config parameters

Put here everything that could be needed if this notebook

In [ ]:
config = AttrDict(
    use_wandb = True, # Whether to use or not wandb for experiment tracking
    wandb_group = None, # Whether to group this run in a wandb group
    dr_artifact_name = 'JNK:v18', # * Set to None for using the default one (DCAE)
    dcae_run_path= 'pacmel/timecluster-extension/48afdwny', #'pacmel/timecluster-extension/red8g4bp' # *
    n_neighbors = 15, #UMAP
    min_dist = 0.1, #UMAP
    metric = 'euclidean', #UMAP
    verbose = True # UMAP
)

This model needs to restore the model fitted in the notebook `02_DCAE`, as well as the data and configuration

Taking the best run from a sweep. 

## Run

In [ ]:
run_dr = wandb.init(entity="pacmel",
                    project="timecluster-extension", 
                    group=config.wandb_group,
                    allow_val_change=True, 
                    job_type='dimensionality_reduction', 
                    mode='online' if config.use_wandb else 'offline',
                    config=config)
config_dr = wandb.config # Object for storing hyperparameters

wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Restore a DCAE model and its associated configuration

In [ ]:
dcae_model = load_model(wandb.restore('model-best.h5', replace=True, run_path=config_dr.dcae_run_path).name)
with open(wandb.restore('config.yaml', replace=True, run_path=config_dr.dcae_run_path).name) as file:
    dcae_config = load(file, FullLoader)

Restore the dataset artifact used for creating the DCAE. Even if we do not compute the dimensionality reduction over this dataset,
we need to know the metadata of the dataset of the DCAE, to check that it matches with the dataset that we want to reduce.

In [ ]:
dcae_artifact_type = dcae_config.get('ds_train_artifact_type').get('value')
dcae_artifact_name = dcae_config.get('ds_train_artifact_name').get('value')
dcae_artifact_digest = dcae_config.get('ds_train_artifact_digest').get('value')
dcae_artifact_type, dcae_artifact_name, dcae_artifact_digest

('dataset', 'JNK:v17', 'bc2c682a76fd28127848efd52c6a9d87')

In [ ]:
dcae_artifact = run_dr.use_artifact(dcae_artifact_name)
dcae_artifact.metadata

{'TS': {'ed': '2019-06-23 23:59:55',
  'sd': '2019-06-01 00:00:00',
  'freq': '<5 * Seconds>',
  'hash': '2649617295588873929',
  'vars': ['RCD_AverageThree-phaseCurrent',
   'LCD_AverageThree-phaseCurrent',
   'LP_AverageThree-phaseCurrent',
   'LHD_LeftHaulageDrive(tractor)Temperature(gearbox)',
   'RHD_RightHaulageDrive(tractor)Temperature(gearbox)',
   'LA_LeftArmTemperature',
   'RA_RightArmTemperature',
   'SM_DailyRouteOfTheShearer',
   'SM_TotalRoute',
   'LHD_EngineCurrent',
   'RHD_EngineCurrent',
   'RCD_BearingTemperature',
   'SM_ShearerSpeed',
   'SM_ShearerLocation',
   'SM_ShearerMoveInLeft',
   'SM_ShearerMoveInRight'],
  'n_vars': 16,
  'created': 'from-df',
  'n_samples': 397440,
  'normalization': {'stds': {'SM_TotalRoute': 21.992641402532836,
    'SM_ShearerSpeed': 2.724568932534116,
    'LHD_EngineCurrent': 21.62938860961895,
    'RHD_EngineCurrent': 21.0185248258138,
    'SM_ShearerLocation': 79.54958101088538,
    'SM_ShearerMoveInLeft': 0.23984552514921795,
   

Now we specify the dataset artifact that we want to use for the reduction. If no artifact is defined, the artifact to reduce will be the one used for training the DCAE.

In [ ]:
if config_dr.dr_artifact_name is not None:
    dr_artifact = run_dr.use_artifact(config_dr.dr_artifact_name, type='dataset')
else:
    dr_artifact = dcae_artifact

Now we need to check whether the artifact that is going to be used fort the dimensionality reduction matches the artifact used to train the DCAE. Matching means having the same number of variables, the same window size and stride, and the same frequency.

In [ ]:
#export
def check_compatibility(dr_ar:TSArtifact, dcae_ar:TSArtifact):
    "Function to check that the artifact used by the DCAE and the artifact that is \
    going to be passed through the DR are compatible"
    try:
        # Check that both artifacts have the same variables
        chk_vars = dr_ar.metadata['TS']['vars'] == dcae_ar.metadata['TS']['vars']
        # Check that both artifacts have the same freq
        chk_freq = dr_ar.metadata['TS']['freq'] == dcae_ar.metadata['TS']['freq']
        # Check that the dr artifact is not normalized (not normalized data has not the key normalization)
        chk_norm = dr_ar.metadata['TS'].get('normalization') is None
        # Check that the dr artifact has not missing values
        chk_miss = dr_ar.metadata['TS']['has_missing_values'] == "False"
        # Check all logical vars.
        if chk_vars and chk_freq and chk_norm and chk_miss:
            print("Artifacts are compatible.")
        else:
            raise Exception
    except Exception as e:
        print("Artifacts are not complatible.")
        raise e
    return None


In [ ]:
#export
def normalize_artifact(artifact:TSArtifact, artifact_norm:TSArtifact):
    "Function to normalize an artifact with the normalization parameters of another \
    one"
    # Check that artifacts are compatible
    check_compatibility(artifact,artifact_norm)
    # Get the normalization parameters of artifact_norm
    ar_norm_stds = artifact_norm.metadata['TS']['normalization']['stds']
    ar_norm_means = artifact_norm.metadata['TS']['normalization']['means']
    # Transform artifact to pandas and iterate over columns
    artifact_df = artifact.to_df()
    # Iterate over df columns to normalize them with means and stds of artifact_norm
    for column_name in artifact_df:
        artifact_df[column_name] = (artifact_df[column_name] - ar_norm_means[column_name])/ar_norm_stds[column_name]
    
    return artifact_df    


In [ ]:
df = normalize_artifact(dr_artifact, dcae_artifact)
df

Artifacts are compatible.


,RCD_AverageThree-phaseCurrent,LCD_AverageThree-phaseCurrent,LP_AverageThree-phaseCurrent,LHD_LeftHaulageDrive(tractor)Temperature(gearbox),RHD_RightHaulageDrive(tractor)Temperature(gearbox),LA_LeftArmTemperature,RA_RightArmTemperature,SM_DailyRouteOfTheShearer,SM_TotalRoute,LHD_EngineCurrent,RHD_EngineCurrent,RCD_BearingTemperature,SM_ShearerSpeed,SM_ShearerLocation,SM_ShearerMoveInLeft,SM_ShearerMoveInRight
TIMESTAMP,,,,,,,,,,,,,,,,
2019-06-24 00:00:00,-0.626543,-0.638671,-0.684975,-1.261235,-0.258498,-1.400388,-1.420405,-0.981145,1.211546,-0.555264,-0.554665,-1.563860,-0.352158,-0.563821,-0.264938,-0.309463
2019-06-24 00:00:05,-0.626543,-0.633343,-0.684975,-1.261235,-0.258498,-1.400388,-1.420405,-0.984025,1.211546,-0.555264,-0.554665,-1.563860,-0.352158,-0.563821,-0.264938,-0.309463
2019-06-24 00:00:10,-0.626543,-0.638671,-0.684975,-1.261235,-0.258498,-1.400388,-1.420405,-0.984025,1.211546,-0.555264,-0.554665,-1.563860,-0.352158,-0.563821,-0.264938,-0.309463
2019-06-24 00:00:15,-0.626543,-0.633343,-0.684975,-1.261235,-0.258498,-1.400388,-1.420405,-0.984025,1.211546,-0.555264,-0.554665,-1.563860,-0.352158,-0.563821,-0.264938,-0.309463
2019-06-24 00:00:20,-0.626543,-0.644000,-0.684975,-1.261235,-0.258498,-1.400388,-1.420405,-0.984025,1.211546,-0.555264,-0.554665,-1.563860,-0.352158,-0.563821,-0.264938,-0.309463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-28 23:59:40,-0.600187,-0.633343,-0.684975,0.667508,-0.258498,0.479399,0.334986,1.597454,1.984531,-0.555264,-0.554665,-0.099532,-0.352158,0.466982,-0.264938,-0.309463
2019-06-28 23:59:45,-0.600187,-0.633343,-0.684975,0.667508,-0.258498,0.479399,0.334986,1.597454,1.984531,-0.555264,-0.554665,-0.099532,-0.352158,0.466982,-0.264938,-0.309463
2019-06-28 23:59:50,-0.600187,-0.633343,-0.684975,0.667508,-0.258498,0.479399,0.334986,1.597454,1.984531,-0.555264,-0.554665,-0.099532,-0.352158,0.466982,-0.264938,-0.309463


In [ ]:
df.shape

(86401, 16)

In [ ]:
dcae_input = df_slicer(df=df,
                       w=dcae_config.get('w').get('value'),
                       s=dcae_config.get('stride').get('value'))
dcae_input.shape

(12330, 96, 16)

### Getting the latent variables from the trained model

Once the DCAE model is trained, we are interested in the information contained in the **Dense** layer (also called *bottleneck*)for each slice of data. To do that, we have to call the `predict` function on an intermediate model that gets the output of the intermediate layer.

In [ ]:
#export dcae
from tensorflow.keras.models import Model
def get_latent_features(dcae, input_data, bottleneck_ln='latent_features'):
    "Get the activations of the bottleneck layer within the fitted autoencoder `dcae` (a Keras model) \
    for the input data `input_data` (a tensor). The name of the bottleneck layer is given in `bottleneck_ln"
    layer_latent_output = dcae.get_layer(bottleneck_ln).output
    intermediate_model = Model(inputs=dcae.input, outputs=layer_latent_output)
    intermediate_prediction = intermediate_model.predict(input_data)
    return intermediate_prediction

In [ ]:
intermediate_prediction = get_latent_features(dcae_model, dcae_input, 'latent_features')
intermediate_prediction.shape

(12330, 96)

## Dimensionality reduction using UMAP

We use DR techniques to provide an alternative view for users to visually analyze and explore the time-series data. The algorithm UMAP shows its high competitiveness compared to t-SNE. t-SNE suffers from some limitations such as loss of large-scale information (the inter-cluster relationships). UMAP has a faster runtime and provides better scaling which helps to gain a meaningful organization of clusters, outliers and the preservation of continuums compared to t-SNE

For this part of the implementation, the package [umap-learn](https://github.com/lmcinnes/umap) is used. The input of the algoritm is the $n \times \delta$ that contains, for each slice of the time series, the corresponding $\delta$ latent features given by the DCAE.

The hyperparameters of UMAP are given values by default here. If the value has been already set previously, that means this notebook is being called from a wandb sweep, and we must use the value that the sweep is bringing.

In [ ]:
umap_params = {
    'n_neighbors' : config_dr.n_neighbors,
    'min_dist' : config_dr.min_dist,
    'metric' : config_dr.metric,
    'verbose': True
}

In [ ]:
# config_dr.update(umap_params, allow_val_change=True)

Finall, we gather all this functionality into a function for future use

In [ ]:
#export
import warnings
from numba.core.errors import NumbaPerformanceWarning
@delegates(umap.umap_.UMAP)
def fget_UMAP_embeddings(input_data, **kwargs):
    "Compute the embeddings of `input_data` using UMAP, with a configuration contained in `**kwargs`. \
    Returns also information of the reducer."
    warnings.filterwarnings("ignore", category=NumbaPerformanceWarning) # silence NumbaPerformanceWarning
    reducer = umap.UMAP(**kwargs)
    reducer.fit(input_data)
    embeddings = reducer.transform(input_data)
    return (embeddings, reducer)

In [ ]:
%%time
embeddings, reducer = fget_UMAP_embeddings(intermediate_prediction, **umap_params)
embeddings.shape

UMAP(dens_frac=0.0, dens_lambda=0.0, verbose=True)
Construct fuzzy simplicial set
Fri May 21 14:40:05 2021 Finding Nearest Neighbors
Fri May 21 14:40:05 2021 Building RP forest with 11 trees
Fri May 21 14:40:09 2021 NN descent for 14 iterations
	 1  /  14
	 2  /  14
	 3  /  14
	 4  /  14
	Stopping threshold met -- exiting after 4 iterations
Fri May 21 14:40:25 2021 Finished Nearest Neighbor Search
Fri May 21 14:40:27 2021 Construct embedding
	completed  0  /  200 epochs
	completed  20  /  200 epochs
	completed  40  /  200 epochs
	completed  60  /  200 epochs
	completed  80  /  200 epochs
	completed  100  /  200 epochs
	completed  120  /  200 epochs
	completed  140  /  200 epochs
	completed  160  /  200 epochs
	completed  180  /  200 epochs
Fri May 21 14:40:37 2021 Finished embedding
CPU times: user 4min 52s, sys: 48.7 s, total: 5min 41s
Wall time: 32.4 s


(12330, 2)

In [ ]:
test_eq(embeddings.shape, (intermediate_prediction.shape[0], reducer.n_components))

In [ ]:
# Test if the reducer contains the items of the configuration object
test_eq(all(item in reducer.get_params().items() for item in dict(umap_params).items()), True)

Save the embeddings as an artifact

In [ ]:
from timecluster_extension.utils import ReferenceArtifact

In [ ]:
embeddings_ar = ReferenceArtifact(obj=embeddings, name='embeddings')
embeddings_ar.metadata, embeddings_ar.manifest.entries.values()

({'ref': {'hash': '-7137006075972365459', 'type': "<class 'numpy.ndarray'>"}},
 dict_values([<ManifestEntry ref: file:///home/victor/data/PACMEL-2019/wandb_artifacts/-7137006075972365459/-7137006075972365459>]))

In [ ]:
run_dr.log_artifact(embeddings_ar)

In [ ]:
embeddings_ar.digest

ValueError: Must call wait() before accessing logged artifact properties

In [ ]:
config_dr.update(
    {
          'emb_artifact_type': embeddings_ar.type,
          'emb_artifact_name': embeddings_ar.name,
          'emb_artifact_digest': embeddings_ar.digest
    }, 
    allow_val_change=True)

## Create Precompudet Clusters

In order to integrate precomputed clusters into the embedding space, it's necessary to log artifacts that include the labels of the newly created clusters. 

The cluster creation process is presented below. This creation procedure can be modified according to specific needs. However, the structure of the new artifact must be preserved (it must be a numpy.ndarray and the number of elements must be equal to the number of points in the embedding space).

In [ ]:
print(f'HDBSCAN supported metrics: {list(hdbscan.dist_metrics.METRIC_MAPPING.keys())}')

In [ ]:
# Define HDBSCAN parameters
hdbscan_kwargs = {
    'min_cluster_size' : 100,
    'min_samples' : 15,
    'cluster_selection_epsilon' : 0.08,
}
metric_kwargs = {
    'metric' : 'jaccard'
}

In [ ]:
# Create clusters using HDBSCAN
clusters = hdbscan.HDBSCAN(**hdbscan_kwargs, **metric_kwargs).fit(embeddings)
clusters_labels = clusters.labels_
clusters_labels

In [ ]:
# Testing artifact structure 
test_eq_type(type(clusters_labels), np.ndarray)
test_eq(clusters_labels.size, embeddings.shape[0])

In [ ]:
# Create and log 'clusters_labels' artifact
clusters_ar = ReferenceArtifact(obj=clusters_labels, name='clusters_labels')
clusters_ar.metadata, clusters_ar.manifest.entries.values()

In [ ]:
run_dr.log_artifact(clusters_ar, aliases=['hdbscan_jaccard'])

## 2D Visualization 

While the connected scatter plot is a simple visualization technique, it has very specific functions in our approach. Every sliding window is represented as a dot in the plot after the projection process (Fig. 4C, D of the paper). Before labeling, all points have the same color and transparency, and when they are concentrated in one area, the densities are accumulated. Lines are used to connect consecutive points preserving the temporal ordering of the data and allowing the user to see temporal connections (Fig. 4B of the paper). Thus, the point is linked to the previous point (inner) and to the posterior point (outer) as an indication of the flow of time.

In [ ]:
#export
def plot_embeddings(embeddings, umap_params, fig_size = (25,25)):
    "Plot 2D embeddings thorugh a connected scatter plot"
    df_embeddings = pd.DataFrame(embeddings, columns = ['x1', 'x2'])
    fig = plt.figure(figsize=(fig_size[0],fig_size[1]))
    ax = fig.add_subplot(111)
    ax.scatter(df_embeddings['x1'], df_embeddings['x2'], marker='o', facecolors='none', edgecolors='b', alpha=0.1)
    ax.plot(df_embeddings['x1'], df_embeddings['x2'], alpha=0.5, picker=1)
    plt.title('DR params -  n_neighbors:{:d} min_dist:{:f} metric:{:s}'.format(umap_params['n_neighbors'],umap_params['min_dist'],umap_params['metric']))
    return ax

In [ ]:
embeddings_plot = plot_embeddings(embeddings, umap_params)

Log this plot as part of the current wandb run

In [ ]:
#hide

# Get the figure of the embedding plot, and save it on thea wandb run.
run_dr.log({"img": [wandb.Image(embeddings_plot.get_figure(), caption="dr_embedding_plot")]})

#run_dr.log({'embeddings_plot': embeddings_plot})

In [ ]:
#hide
run_dr.finish()

# Explainability with SHAP (future work)

In [ ]:
# %matplotlib inline

# fig = plt.figure(figsize=(10,10))
# ax = fig.add_subplot(111)

# ax.scatter(df_embeddings['x1'], df_embeddings['x2'], marker='o', facecolors='none', edgecolors='b', alpha=0.1)
# ax.plot(df_embeddings['x1'], df_embeddings['x2'], alpha=0.5, picker=1)
# ax.set_title('Select the point you want to visualize as a time window in the original space')

Make the plot interactive to allow selection of subsets of the plot

In [ ]:
# selected_indices = None
# selected_points = None

# def onpick(event):
#     global selected_points
#     thisline = event.artist
#     xdata = thisline.get_xdata()
#     ydata = thisline.get_ydata()
#     global selected_indices
#     selected_indices = event.ind
#     selected_points = tuple(zip(xdata[selected_indices], ydata[selected_indices]))
#     print('onpick points (first):', selected_points[0])

# fig.canvas.mpl_connect('pick_event', onpick)

# plt.show()
# fig.tight_layout()
# fig.savefig(f'../img/w={w}.png')

## Hyperparameter tuning for the dimensionality reduction

There are a number of parameters that can be set for the UMAP algorithm. The major 
ones are `n_neighbors` and `min_dist`. Thus, we will carry out a hyperparameter 
sweep in Weights and Biases for these two parameters. Note that there is no objective
way of deciding that some embeddings are better than others. Thus, we must rely on our
intuition by visualizing the 2D plots of each of the runs in the sweep.

The first thing we need is gather all the pipeline of the previous section into a function


## Linking back points of the 2D projection to the original time series

The variable `selected_points` and `ind` contain an array of the points and indices selected in the previous 2D projection. We will take the first of them (there can be many selected points with just one click), and use its index to get the corresponding time window of the original space.

In [ ]:
# selected_window = input_data[df_embeddings.sample(n=1).index][0] if selected_indices is None else input_data[selected_indices[0]]
# selected_window

### Visualizing all the variables in the time window (default)

In [ ]:
# %matplotlib inline
# g = sns.FacetGrid(df_output_tidy, col="variable", col_wrap=3, aspect=2)
# g = g.map(plt.plot, "timestamp", "value")

### Contribution: Visualize only the most relevant variables

In high dimensional time series, not only is interesting to see the window associated to a point in the 2D space, but also it is extremely important to spot which variables are mainly causing that the window is positioned in that point of the 2D space.

Since UMAP does not provide capabilities to understand feature importance, there are [different ways](https://stats.stackexchange.com/questions/438025/understand-important-features-in-umap) to tackle this problem:

1. Use another dimensionality reduction technique that provides importance, such as [sparse PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.SparsePCA.html)

2. Create a surrogate model on top of the inputs/output of UMAP and explain it using XAI techniques. We will try here this option.

The idea is to have a surrogate model that takes the multivariate time series as input and produces the associated points in the 2D space as ouput. Since we already have a Deep Convolutional Autoencoder (DCAE) that takes a multivariate time series as input, and it contains the latent features that represent that input, we can use it for the surrogate. We will use the intermediate model that goes from the input to the layer containing the latent space, and then add a `Dense` layer with 2 units and linear activation.

In [ ]:
#export
# def train_surrogate_model(dcae, embeddings, lat_ln='latent_features'):
#     "Train a surrogate model that learns the `embeddings` from the latent features contained in the layer \
#     `lat_ln` of a previously trained Deep Convolutional AutoEncoder `dcae`"
#     x = dcae.get_layer(lat_ln).output
#     x = Dense(units=embeddings.shape[1], activation='linear')(x)
#     surrogate_model = Model(dcae.input, x)
#     l_nms = [layer.name for layer in surrogate_model.layers]
#     layer_idx = l_nms.index(lat_ln)
#     # The layers that are already trained from the autoencoder must be `frozen`
#     for layer in surrogate_model.layers[:layer_idx]:
#         layer.trainable = False
#     return surrogate_model

In [ ]:
# sm = train_surrogate_model(m, embeddings, lat_ln='latent_features')

In [ ]:
# test.equals(sm.input.shape, m.input.shape)
# test.equals(sm.output.shape[1], embeddings.shape[1])
# l_nms = [layer.name for layer in sm.layers]
# layer_idx = l_nms.index('latent_features')
# test.all_equal([layer.trainable for layer in sm.layers], \
#                np.repeat([False, True], [layer_idx + 1, len(sm.layers) -1 -layer_idx]))

Configure the training hyperparameters

In [ ]:
# loss = 'mean_squared_error'
# opt = 'adam'
# bs = 100
# epochs = 10
# val = .2

In [ ]:
# sm.output_shape

In [ ]:
# input_data.shape

In [ ]:
# embeddings.shape

In [ ]:
# sm.fit(x=input_data, y=embeddings, batch_size=bs, validation_split=val, epochs=epochs, callbacks=[WandbCallback()])

In [ ]:
# import innvestigate

In [ ]:
# analyzer = innvestigate.create_analyzer("gradient", intermediate_model)

In [ ]:
# asd= innvestigate.create_analyzer("gradient", m)

In [ ]:
# from keras.datasets import mnist

In [ ]:
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# x_train.shape

In [ ]:
# input_data[np.random.choice(input_data.shape[0], 100, replace=False)].shape

In [ ]:
# import shap

In [ ]:
# background = input_data[np.random.choice(input_data.shape[0], 100, replace=False)]
# e = shap.DeepExplainer(intermediate_model, background)

In [ ]:
# shap_values = e.shap_values(input_data[0:4])

In [ ]:
# shap_values[0][0]